<a href="https://colab.research.google.com/github/aswindcu2025/churn_prediction_telecom_ds/blob/areen/Model_building_%26_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv("/content/preprocessed_df.csv")

In [ ]:
df.head()

,tenure_scaled,MonthlyCharges_scaled,TotalCharges_scaled,gender,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn
0,-0.463037,0.002935,-0.416007,0,0,0,1,1,0,1,1,0,1,0,0,1,1,0,3,0
1,0.880735,1.078118,1.257246,0,0,0,0,1,1,2,0,1,0,0,1,1,2,1,0,0
2,-1.277445,-1.373033,-0.995434,1,0,1,0,0,2,1,0,0,0,0,0,0,0,1,2,1
3,-1.155283,0.180747,-0.900800,1,0,0,0,1,0,2,0,0,0,0,0,0,0,1,2,1
4,-1.318165,-0.095111,-1.005780,1,0,0,1,1,1,1,1,1,0,1,0,0,2,1,0,0


In [ ]:
X = df.drop('Churn', axis=1)
y = df['Churn']

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(X,y,test_size=.2,random_state=42)

In [ ]:
model_params= {
    'lr': {
        'model': LogisticRegression(),
        'params':{
            'C': [1,5,10]

            }
    },
    'DTC': {
        'model':DecisionTreeClassifier(),
        'params':{
            'criterion': ["gini", "entropy"],
            'max_depth' : [2, 3, 5, 10, 20, None],
            'min_samples_split' : [2, 5, 10, 20, 50],
            'splitter' : ["best", "random"]

            }
    },
    'rfc':{
        'model': RandomForestClassifier(),
        'params':{
            'n_estimators': [100, 200, 300, 500],
            'max_depth' : [None, 10, 20, 30, 50],
            'max_features' :['sqrt', 'log2', None]

            }
    },
    'lgbm': {
    'model': LGBMClassifier(),
    'params': {
        'num_leaves': [31, 50, 70],
        'learning_rate': [0.01, 0.05, 0.1],
        'n_estimators': [50,100,150,200],
        'subsample': [0.7, 1.0],
        'colsample_bytree': [0.7, 1.0],
        'max_depth': [-1, 5, 10]

        }
    },
    'bagging':{
        'model': BaggingClassifier(),
        'params': {
            'estimator': [DecisionTreeClassifier(),DecisionTreeClassifier(max_depth=5),DecisionTreeClassifier(max_depth=10)],
            'n_estimators':[50,100,150,200],
            'max_samples':[0.25,0.5, 0.7, 0.8],
            'max_features' : [0.5, 0.7, 1.0]

        }
    }
}

In [ ]:
from sklearn import model_selection
score=[ ]

for model_name, mp in model_params.items():
  gscv=GridSearchCV(mp['model'],mp['params'],cv=5,return_train_score=False)

  gscv.fit(xtrain,ytrain)

  score.append({'model':model_name,
                'best_score': gscv.best_score_,
                'best_params':gscv.best_params_})

Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [ ]:
pd.DataFrame(score)

,model,best_score,best_params
0,lr,0.775227,{'C': 5}
1,DTC,0.781118,"{'criterion': 'gini', 'max_depth': 10, 'min_sa..."
2,rfc,0.835196,"{'max_depth': 20, 'max_features': 'sqrt', 'n_e..."
3,lgbm,0.830363,"{'colsample_bytree': 1.0, 'learning_rate': 0.1..."
4,bagging,0.825680,"{'estimator': DecisionTreeClassifier(), 'max_f..."


In [ ]:
best_model_info = max(score, key=lambda x: x['best_score'])

print("Best Model Name:", best_model_info['model'])
print("Best Score:", best_model_info['best_score'])
print("Best Parameters:", best_model_info['best_params'])

Best Model Name: rfc
Best Score: 0.8351963746223566
Best Parameters: {'max_depth': 20, 'max_features': 'sqrt', 'n_estimators': 200}


In [ ]:
model_class = model_params[best_model_info['model']]['model'].__class__
best_params = best_model_info['best_params']
best_model = model_class(**best_params)

In [ ]:
best_model.fit(xtrain, ytrain)

RandomForestClassifier(max_depth=20, n_estimators=200)

In [ ]:
y_pred = best_model.predict(xtest)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Accuracy:", accuracy_score(ytest, y_pred))
print("\nClassification Report:\n", classification_report(ytest, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(ytest, y_pred))

Accuracy: 0.8306659783169851

Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.83      0.84      1063
           1       0.80      0.84      0.82       874

    accuracy                           0.83      1937
   macro avg       0.83      0.83      0.83      1937
weighted avg       0.83      0.83      0.83      1937


Confusion Matrix:
 [[879 184]
 [144 730]]
